API SPOTIPY

In [10]:
import sys
print(sys.executable)

c:\Users\apdue\anaconda3\python.exe


In [11]:
import sys
!{sys.executable} -m pip install spotipy

In [12]:
import spotipy

In [13]:
from spotipy.oauth2 import SpotifyClientCredentials

In [14]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
import subprocess
subprocess.check_call(["pip", "install", "mysql-connector"])

0

In [16]:
import sys
!{sys.executable} -m pip install mysql-connector-python

In [17]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import time

# ---- SPOTIFY ----
CLIENT_ID = "ce410d235c6444c9928df476e4270803"
CLIENT_SECRET = "af63ec6e05084a4d9edc175a055bd4f2"

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

# ---- PARÁMETROS ----
generos = ['pop', 'rock', 'jazz', 'reggaeton']
anio_inicio, anio_final = 2019, 2024
max_canciones = 500
todos_los_resultados = []

# ---- RECOLECCIÓN DE CANCIONES ----
for genero in generos:
    print(f"Buscando canciones para el género {genero}...")
    canciones_obtenidas = 0
    offset = 0
    limite = 50

    while canciones_obtenidas < max_canciones:
        query = f'genre:"{genero}" year:{anio_inicio}-{anio_final}'
        respuesta = sp.search(q=query, type='track', limit=limite, offset=offset, market="ES")
        tracks = respuesta.get('tracks', {}).get('items', [])

        if not tracks:
            break

        for track in tracks:
            todos_los_resultados.append({
                'name_track': track['name'],
                'name_artist': track['artists'][0]['name'],
                'genre': genero,
                'type': track['type'],
                'year': track['album']['release_date'][:4]
            })

        canciones_obtenidas += len(tracks)
        offset += limite
        time.sleep(0.2)

    print(f"Se obtuvieron {canciones_obtenidas} canciones para {genero}")

print(f"Total de canciones obtenidas: {len(todos_los_resultados)}")

# ---- CONVERTIR A DATAFRAME ----
df_spotipy = pd.DataFrame(todos_los_resultados)

# ---- MYSQL ----
user = 'root'
password = 'AlumnaAdalab'
host = '127.0.0.1'
db_name = 'prueba_music_stream_proyecto'

# Crear la base de datos si no existe
try:
    cnx = mysql.connector.connect(user=user, password=password, host=host)
    cursor = cnx.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Base de datos '{db_name}' creada o ya existente.")
    cursor.close()
    cnx.close()
except mysql.connector.Error as err:
    print(f"Ocurrió un error creando la base de datos: {err}")

# Conectar con SQLAlchemy para pasar DataFrame directamente
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{db_name}')

# Guardar DataFrame en MySQL
try:
    df_spotipy.to_sql('canciones', engine, if_exists='replace', index=False)
    print(f"Se insertaron {len(df_spotipy)} canciones en la tabla 'canciones'.")
except Exception as e:
    print(f"Ocurrió un error al guardar en SQL: {e}")

Buscando canciones para el género pop...
Se obtuvieron 500 canciones para pop
Buscando canciones para el género rock...
Se obtuvieron 500 canciones para rock
Buscando canciones para el género jazz...
Se obtuvieron 500 canciones para jazz
Buscando canciones para el género reggaeton...
Se obtuvieron 120 canciones para reggaeton
Total de canciones obtenidas: 1620
Base de datos 'prueba_music_stream_proyecto' creada o ya existente.
Se insertaron 1620 canciones en la tabla 'canciones'.


In [18]:
# Mostrar las primeras 10 filas
print(df_spotipy.head(10))

# Mostrar información general del DataFrame
print(df_spotipy.info())

# Mostrar algunas estadísticas básicas (si aplica)
print(df_spotipy.describe(include='all'))

# Ver todos los artistas únicos
print(df_spotipy['name_artist'].unique())

# Ver el número total de canciones y artistas
print(f"Total de canciones: {len(df_spotipy)}")
print(f"Total de artistas únicos: {df_spotipy['name_artist'].nunique()}")

           name_track    name_artist genre   type  year
0          La Morocha        Luck Ra   pop  track  2023
1          La Morocha        Luck Ra   pop  track  2024
2     Todos los Besos          Siloé   pop  track  2023
3   Mon Amour - Remix         zzoilo   pop  track  2021
4     Algo Sencillito   Marta Santos   pop  track  2023
5             Contigo   Marta Santos   pop  track  2024
6                APT.           ROSÉ   pop  track  2024
7  BIRDS OF A FEATHER  Billie Eilish   pop  track  2024
8             Soltera        Shakira   pop  track  2024
9      That’s So True  Gracie Abrams   pop  track  2024
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620 entries, 0 to 1619
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name_track   1620 non-null   object
 1   name_artist  1620 non-null   object
 2   genre        1620 non-null   object
 3   type         1620 non-null   object
 4   year         1620 non-null 

API LAST.FM 

In [19]:
import requests
import pandas as pd
import re
import time

# ---- Lista de artistas únicos desde Spotify ----
lista_artistas = df_spotipy['name_artist'].dropna().unique().tolist()

# ---- Función para limpiar nombres de artistas ----
def limpiar_nombre_artista(nombre):
    nombre = nombre.lower()
    nombre = re.sub(r"\(.*?\)|feat\. .*|ft\. .*", "", nombre)  # quitar feat, ft, paréntesis
    nombre = re.sub(r"[^a-z0-9\s]", "", nombre)  # quitar caracteres especiales
    nombre = nombre.strip()
    return nombre

# Lista limpia de artistas
lista_artistas_limpia = [limpiar_nombre_artista(a) for a in lista_artistas]

# ---- Configuración Last.fm ----
API_KEY = "4c376bffe433789718d87658d629efca"
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

def obtener_datos_lastfm_artistas(lista_artistas, max_tracks=5, max_similares=5, sleep_time=0.2):
    registro_artistas = []

    for artista_nombre in lista_artistas:
        try:
            # --- Información general del artista ---
            resp_info = requests.get(BASE_URL, params={
                "method": "artist.getinfo",
                "artist": artista_nombre,
                "api_key": API_KEY,
                "autocorrect": 1,
                "format": "json"
            }, timeout=10)
            resp_info.raise_for_status()
            datos = resp_info.json().get("artist")
            if not datos:
                continue

            stats = datos.get("stats", {})
            bio = datos.get("bio", {}).get("summary", "")
            url = datos.get("url", "")
            tags_list = datos.get("tags", {}).get("tag", [])
            if isinstance(tags_list, dict):
                tags_list = [tags_list]
            tags = [t.get("name","") for t in tags_list]

            # --- Top tracks ---
            top_tracks_info = []
            resp_tracks = requests.get(BASE_URL, params={
                "method": "artist.gettoptracks",
                "artist": artista_nombre,
                "api_key": API_KEY,
                "autocorrect": 1,
                "format": "json",
                "limit": max_tracks
            }, timeout=10)
            resp_tracks.raise_for_status()
            tracks_json = resp_tracks.json().get("toptracks", {}).get("track", [])
            if isinstance(tracks_json, dict):
                tracks_json = [tracks_json]

            for track in tracks_json:
                track_name = track.get("name","")
                track_info = {"name": track_name}
                try:
                    resp_track = requests.get(BASE_URL, params={
                        "method": "track.getinfo",
                        "artist": artista_nombre,
                        "track": track_name,
                        "api_key": API_KEY,
                        "autocorrect": 1,
                        "format": "json"
                    }, timeout=10)
                    resp_track.raise_for_status()
                    track_data = resp_track.json().get("track", {})
                    tags_track = track_data.get("toptags", {}).get("tag", [])
                    if isinstance(tags_track, dict):
                        tags_track = [tags_track]
                    tags_cancion = [tg.get("name","") for tg in tags_track]
                    track_info.update({
                        "album": track_data.get("album", {}).get("title",""),
                        "duration_s": int(track_data.get("duration") or 0)//1000,
                        "tags": tags_cancion,
                        "wiki": track_data.get("wiki", {}).get("summary","")
                    })
                except:
                    pass
                top_tracks_info.append(track_info)

            # --- Artistas similares ---
            similares = []
            resp_sim = requests.get(BASE_URL, params={
                "method": "artist.getsimilar",
                "artist": artista_nombre,
                "api_key": API_KEY,
                "autocorrect": 1,
                "format": "json",
                "limit": max_similares
            }, timeout=10)
            resp_sim.raise_for_status()
            similar_json = resp_sim.json().get("similarartists", {}).get("artist", [])
            if isinstance(similar_json, dict):
                similar_json = [similar_json]
            similares = [s.get("name","") for s in similar_json]

            # --- Guardar info del artista ---
            registro_artistas.append({
                "Artist": artista_nombre,
                "Listeners": int(stats.get("listeners",0) or 0),
                "Playcount": int(stats.get("playcount",0) or 0),
                "URL": url,
                "Biography": bio,
                "Tags": tags,
                "TopTracks": top_tracks_info,
                "SimilarArtists": similares
            })

            time.sleep(sleep_time)  # respetar rate limit

        except Exception as e:
            print(f"Error con artista {artista_nombre}: {e}")
            continue

    return pd.DataFrame(registro_artistas)


  

In [20]:
# Tomar solo los primeros 5 artistas para test rápido. FUNCIONA
df_lastfm = obtener_datos_lastfm_artistas(lista_artistas_limpia[:5], max_tracks=2, max_similares=2)

print(df_lastfm.head())
print(df_lastfm.shape)

         Artist  Listeners  Playcount                                     URL  \
0       luck ra     103603    2854164       https://www.last.fm/music/Luck+Ra   
1          silo      57557     576438          https://www.last.fm/music/Silo   
2        zzoilo     147239    1812845        https://www.last.fm/music/zzoilo   
3  marta santos      16048     195181  https://www.last.fm/music/Marta+Santos   
4           ros       9676     114557           https://www.last.fm/music/Ros   

                                           Biography  \
0  Juan Facundo Almenara, conocido por su nombre ...   
1  There are at least 5 acts using the name Silo:...   
2  Zoilo Tuñón Rosa, más conocido como Zzoilo, es...   
3   <a href="https://www.last.fm/music/Marta+Sant...   
4  Rapcore, Nu Metal band from Japan. ROS Formed ...   

                                                Tags  \
0  [white rapper, bachata, merengue, racist, chil...   
1  [post-rock, trip-hop, hypnotic, experimental, ...   
2  [span

Hemos tenido que utilizar concurrencia de hilos para poder ejecutar el código, pues se hacía muy lento

In [21]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Función que procesa un solo artista (envoltorio)
def obtener_datos_lastfm_artista(artista):
    return obtener_datos_lastfm_artistas([artista], max_tracks=5, max_similares=5, sleep_time=0)

# ---- Ejecutar en paralelo ----
resultados = []
with ThreadPoolExecutor(max_workers=10) as executor:  # prueba 10-15 para más velocidad
    futures = {executor.submit(obtener_datos_lastfm_artista, artista): artista for artista in lista_artistas_limpia}
    
    for future in as_completed(futures):
        artista = futures[future]
        try:
            df_tmp = future.result()
            if not df_tmp.empty:
                resultados.append(df_tmp)
        except Exception as e:
            print(f"Error procesando artista {artista}: {e}")

# ---- Unir todos los resultados ----
df_lastfm = pd.concat(resultados, ignore_index=True) if resultados else pd.DataFrame()

# ---- Resultado ----
print(df_lastfm.head())
print(df_lastfm.shape)

               Artist  Listeners  Playcount  \
0       billie eilish    3888267  738118865   
1             shakira    4560365  164675228   
2       gracie abrams    1629406  247336933   
3              zzoilo     147239    1812845   
4  daviles de novelda      21100     196828   

                                            URL  \
0       https://www.last.fm/music/Billie+Eilish   
1             https://www.last.fm/music/Shakira   
2       https://www.last.fm/music/Gracie+Abrams   
3              https://www.last.fm/music/zzoilo   
4  https://www.last.fm/music/Daviles+de+Novelda   

                                           Biography  \
0  Billie Eilish Pirate Baird O'Connell (born Dec...   
1  Shakira Isabel Mebarak Ripoll is a Colombian s...   
2  Gracie Abrams (born September 7, 1999) is an A...   
3  Zoilo Tuñón Rosa, más conocido como Zzoilo, es...   
4   <a href="https://www.last.fm/music/Daviles+de...   

                                                Tags  \
0  [pop, indie po

In [22]:
import json
from sqlalchemy import create_engine, VARCHAR, BIGINT, TEXT
import mysql.connector

# ---- Parámetros de conexión ----
user = 'root'
password = 'AlumnaAdalab'
host = '127.0.0.1'
db_name = 'prueba_music_stream_proyecto'

# ---- Crear la base de datos si no existe ----
try:
    cnx = mysql.connector.connect(user=user, password=password, host=host)
    cursor = cnx.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Base de datos '{db_name}' creada o ya existente.")
    cursor.close()
    cnx.close()
except mysql.connector.Error as err:
    print(f"Ocurrió un error creando la base de datos: {err}")

# ---- Convertir listas/diccionarios a JSON strings ----
df_lastfm_copy = df_lastfm.copy()
for col in ['Tags', 'TopTracks', 'SimilarArtists']:
    df_lastfm_copy[col] = df_lastfm_copy[col].apply(lambda x: json.dumps(x, ensure_ascii=False))

# ---- Conexión a MySQL usando SQLAlchemy ----
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{db_name}')

# ---- Guardar DataFrame en SQL ----
try:
    df_lastfm_copy.to_sql(
        'artistas',
        engine,
        if_exists='replace',
        index=False,
        dtype={
            'Artist': VARCHAR(100),
            'Listeners': BIGINT,
            'Playcount': BIGINT,
            'URL': VARCHAR(255),
            'Biography': TEXT,
            'Tags': TEXT,
            'TopTracks': TEXT,
            'SimilarArtists': TEXT
        }
    )
    print("Datos guardados correctamente en la tabla 'artistas'.")
except Exception as e:
    print(f"Ocurrió un error al guardar en SQL: {e}")


Base de datos 'prueba_music_stream_proyecto' creada o ya existente.
Datos guardados correctamente en la tabla 'artistas'.
